In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)
import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    get_predict_loader,
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    dataloader_test_by_stock
)
from metrics import calc_spread_return_sharpe
print(torch.__version__)

C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0


# Get data

In [12]:
FOLDER = 'example_test_files'
df = get_data(folder=FOLDER)
# df['Target'] = df['Close'].shift()
# df = df.sort_values(by=['RowId', 'Date'])
print(df.head())
# print(df.groupby('SecuritiesCode').count())

CONT_COLS=['Close', 'Open', 'High', 'Low']
TS_IN_FEATURES = len(CONT_COLS)
CAT_FEATURES = 4 #cat.shape[1]
print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 300
NO_EMBEDDING = 2000 #2 * len(df_train_cat)

                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2021-12-06  20211206_1301            1301  2982.0  2982.0  2965.0  2971.0   
2021-12-06  20211206_1332            1332   592.0   599.0   588.0   589.0   
2021-12-06  20211206_1333            1333  2368.0  2388.0  2360.0  2377.0   
2021-12-06  20211206_1375            1375  1230.0  1239.0  1224.0  1224.0   
2021-12-06  20211206_1376            1376  1339.0  1372.0  1339.0  1351.0   

             Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  
Date                                                                      
2021-12-06     8900               1.0               NaN            False  
2021-12-06  1360800               1.0               NaN            False  
2021-12-06   125900               1.0               NaN            False  
2021-12-06    81100               1.0               NaN            False  
2021-12-06

In [13]:
df['SecuritiesCode'].unique()

array([1301, 1332, 1333, ..., 9993, 9994, 9997], dtype=int64)

In [14]:
df[df['SecuritiesCode'] == 1301]

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
Date,,,,,,,,,,
2021-12-06,20211206_1301,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
2021-12-07,20211207_1301,1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False


In [3]:
stocks = df['SecuritiesCode'].unique()
count = 0
BATCH_SIZE = 512
weight_decay = 0.01

# model = create_model()
# trainer = Trainer(model, optimizer_name='rmsprop', lr=1.3333e-4, weight_decay=weight_decay)
# torch.autograd.set_detect_anomaly(True)

for stock in stocks:
    teast_dataloader = None
    if count > 15:
        break
    print(f'Start training for stock: {stock}')

    test_dataloader = dataloader_test_by_stock(
        df, 
        stock, 
        batch_size=BATCH_SIZE, 
        continous_cols=CONT_COLS
    )


Start training for stock: 1301
                    RowId    Open    High     Low   Close  Volume  \
Date                                                                
2021-12-06  20211206_1301  2982.0  2982.0  2965.0  2971.0    8900   
2021-12-07  20211207_1301  2998.0  3065.0  2990.0  3065.0   19100   

            AdjustmentFactor  ExpectedDividend  SupervisionFlag  Target  \
Date                                                                      
2021-12-06               1.0               NaN            False     NaN   
2021-12-07               1.0               NaN            False     NaN   

            day_of_year  month  day_of_week  
Date                                         
2021-12-06          340     12            6  
2021-12-07          341     12            7  


C:\Users\gilbe\Documents\TokyoStockEx\notebooks\..\preprocessing\preprocess.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat.loc[:, ['RowId']] = txt_transfom.transform()


continuos shape: (2, 9)  categorical shape: (2, 4)


ValueError: cannot reshape array of size 0 into shape (2,1)

# Load model

In [ ]:
trained_model = torch.load('./trained_model.pt')
